# Spaceship Titanic - Eighth Model

## Overview

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, 
the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets 
orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic
collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000
years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

In this competition your task is to predict whether a passenger was transported to an alternate dimension during the 
Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal
records recovered from the ship's damaged computer system.

## File and Data Field Descriptions

### train.csv 

Personal records for about two-thirds (~8700) of the passengers, to be used as training data.

Sure! Here's the code converted to a markdown table:

| Column Name | Description |
|------------- |-------------|
| `PassengerId` | A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always. |
| `HomePlanet` | The planet the passenger departed from, typically their planet of permanent residence. |
| `CryoSleep` | Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins. |
| `Cabin` | The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard. |
| `Destination` | The planet the passenger will be debarking to. |
| `Age` | The age of the passenger. |
| `VIP` | Whether the passenger has paid for special VIP service during the voyage. |
| `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` | Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities. |
| `Name` | The first and last names of the passenger. |
| `Transported` | Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict. |

### test.csv

Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. 

Your task is to predict the value of Transported for the passengers in this set.
    
### sample_submission.csv

A sample submission file in the correct format.

| Column Name | Description |
|------------- |-------------|
| `PassengerId` | A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always. |
| `Transported` | Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict. |

In [1]:
model='eighth'

In [2]:
# Library imports

# Data wrangling
import pandas as pd
import numpy as np
import missingno
from collections import Counter

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

## Load Data

In [3]:
# Load the data - to pandas dataframes

test_df = pd.read_csv('./inputs/test.csv')
test_idx = test_df['PassengerId']
train_df = pd.read_csv('./inputs/train.csv')

## Drop unrequired features & add engineered ones

In [4]:
# split name into components
def split_name(name:str):
    if pd.isnull(name): # protect against empty values
        return (np.nan, np.nan, np.nan)
    parts = name.split(' ')
    first = parts[0]
    last = parts[-1]
    return (first, last)

# split cabin into components
def split_cabin(name:str):
    if pd.isnull(name): # protect against empty values
        return (np.nan, np.nan, np.nan)
    parts = name.split('/')
    deck = parts[0]
    side = parts[-1]
    number = ' '.join(parts[1:-1])
    return (deck, number, side)

# This function will drop and add required features
def feature_manipulation(d : pd.DataFrame) -> pd.DataFrame:
    # print the shape of the provided dataframe
    print("Before: ", d.shape)
    # Clone the provided dataframe
    df = d.copy()

    _, df['Name'] = zip(*df['Name'].map(split_name))
    
    # Location
    df['Deck'], _, df['Side'] = zip(*df['Cabin'].map(split_cabin))
    df['DeckSide'] = df['Deck'] + df['Side'] # Combine Deck and Side

    # Group related columns
    df['Group'] = df['PassengerId'].map(lambda x: x.split('_')[0])
    df['GroupSize'] = df['Group'].map(df['Group'].value_counts())
    
    print("After: ", df.shape)
    return df

In [5]:
train_df = feature_manipulation(train_df)

Before:  (8693, 14)
After:  (8693, 19)


In [6]:
test_df = feature_manipulation(test_df)

Before:  (4277, 13)
After:  (4277, 18)


In [7]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Side,DeckSide,Group,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Ofracculy,False,B,P,BP,0001,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Vines,True,F,S,FS,0002,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Susent,False,A,S,AS,0003,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Susent,False,A,S,AS,0003,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Santantines,True,F,S,FS,0004,1


In [8]:
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Side,DeckSide,Group,GroupSize
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Carsoning,G,S,GS,0013,1
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Peckers,F,S,FS,0018,1
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Unhearfus,C,S,CS,0019,1
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Caltilter,C,S,CS,0021,1
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Harperez,F,S,FS,0023,1


## Handle Missing Data

In the first model we had a complicated approach - here we will just use the median for numeric and mode for non-numeric

In [9]:
# Define a function to print missing data
def print_missing(train: pd.DataFrame, test: pd.DataFrame):
    missing_data = pd.DataFrame({
        'Train Missing': train.isnull().sum(),
        'Test Missing': test.isnull().sum()
    }).sort_values(by='Train Missing', ascending=False)

    print(missing_data)

In [10]:
# Check for missing data before imputing
print_missing(train_df, test_df)

              Train Missing  Test Missing
CryoSleep               217          93.0
ShoppingMall            208          98.0
VIP                     203          93.0
HomePlanet              201          87.0
Name                    200          94.0
Cabin                   199         100.0
Deck                    199         100.0
DeckSide                199         100.0
Side                    199         100.0
VRDeck                  188          80.0
FoodCourt               183         106.0
Spa                     183         101.0
Destination             182          92.0
RoomService             181          82.0
Age                     179          91.0
GroupSize                 0           0.0
PassengerId               0           0.0
Group                     0           0.0
Transported               0           NaN


### Define functions to replace data

In [11]:
def fill_with_median(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].median())
    return df

def fill_with_mode(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].mode()[0])
    return df

def fill_with_mean(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].mean())
    return df

# Here we provide a list of other_features - if a row has a nan in a feature we will fill it with the mean of the other_features in its row 
def fill_with_mean_of_other_features(df, features, other_features):
    for feature in features:
        df[feature] = df[feature].fillna(df[other_features].dropna().mean(axis=1))
    return df

def fill_with_constant(df, features, constant):
    for feature in features:
        df[feature] = df[feature].fillna(constant)
    return df

### Apply simple rules for empty data

In [12]:
def handle_empty_data(df: pd.DataFrame) -> pd.DataFrame:
   
    # fill in missing values with a constant
    df = fill_with_constant(df, ['VIP'], False)
    df = fill_with_constant(df, ['Name', 'Cabin', 'Deck', 'Side', 'DeckSide', 'HomePlanet', 'Destination'], 'Unknown')

    # Fill numerics with median
    df = fill_with_median(df, ['Age'])
    
    # Fill with average of other spend features
    spend_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df = fill_with_mean_of_other_features(df, spend_features, spend_features)
    df = fill_with_constant(df, spend_features, 0)

    # Have a sensible default so leave this as is
    # for the provided dataframe set CryoSleep to False if TotalSpend is greater than 0
    UnknownCryoSpender = (df["CryoSleep"].isnull() | df["CryoSleep"].isna()) & (df[spend_features].dropna().sum(axis=1) > 0)
    df.loc[UnknownCryoSpender, 'CryoSleep'] = False
    df['CryoSleep'] = df['CryoSleep'].fillna('Unknown')

    return df


In [13]:
train_df = handle_empty_data(train_df)
test_df = handle_empty_data(test_df)

In [14]:
# Check for missing data after imputing
print_missing(train_df, test_df)

              Train Missing  Test Missing
Age                       0           0.0
Name                      0           0.0
VIP                       0           0.0
Transported               0           NaN
Spa                       0           0.0
Side                      0           0.0
ShoppingMall              0           0.0
RoomService               0           0.0
PassengerId               0           0.0
HomePlanet                0           0.0
Cabin                     0           0.0
GroupSize                 0           0.0
Group                     0           0.0
FoodCourt                 0           0.0
Destination               0           0.0
DeckSide                  0           0.0
Deck                      0           0.0
CryoSleep                 0           0.0
VRDeck                    0           0.0


### And thats a wrap!

There are now no missing values so we can proceed to Pre-processing

# Data Preprocessing

## Data shape and types at start

In [15]:
# Check the shapes:

print(f'Train: {train_df.shape}')
print(f'Test: {test_df.shape}')

# Check the train and test column data types side by side
pd.concat([train_df.dtypes, test_df.dtypes], axis=1, keys=['Train', 'Test'])

Train: (8693, 19)
Test: (4277, 18)


,Train,Test
PassengerId,object,object
HomePlanet,object,object
CryoSleep,object,object
Cabin,object,object
Destination,object,object
Age,float64,float64
VIP,bool,bool
RoomService,float64,float64
FoodCourt,float64,float64
ShoppingMall,float64,float64


## Drop Unrequired columns

We can now drop the columns we do not want and ensure columns with a mix of str and bool are converted to str columns

In [16]:
# Drop the columns that are not required
def drop_unrequired_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_drop = [
        'Cabin',
        'Name',
        'DeckSide'
    ]
    df.drop(columns_to_drop, axis = 1, inplace = True)
    return df

train_df = drop_unrequired_columns(train_df)
test_df = drop_unrequired_columns(test_df)

# There are some cases where we have used 'Unknown' in boolean columns that have True and False otherwise that led to an error
# complaining about a mix of str and bool so we need to convert the columns to str
for column in train_df.columns:
    if 'Unknown' in train_df[column].unique():
        print(column)
        train_df[column] = train_df[column].astype(str)

HomePlanet
CryoSleep
Destination
Deck
Side


## Pre-process the data for training

In [17]:
# Define the preprocessor
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

# Define pipelines for the features where we wish to bin and OHE
def preprocessor(n_bins=8, strategy='uniform', scaling_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], categorical_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']):
    scaling_pipeline = Pipeline([
        ('scale', MinMaxScaler()),
    ])

    age_pipeline = Pipeline([
        ('scale', RobustScaler())
    ])
    if n_bins > 0:
        age_pipeline = Pipeline([
            ('binning', KBinsDiscretizer(n_bins=n_bins, encode='onehot-dense', strategy=strategy))
        ])

    categorical_pipeline = Pipeline([
        ('onehot', OneHotEncoder())
    ])
    return ColumnTransformer(
        transformers=[
            ('category', categorical_pipeline, categorical_features),
            ('spending', scaling_pipeline, scaling_features),
            ('age', age_pipeline, ['Age'])
        ],
        remainder='passthrough'  # This leaves the rest of the columns untouched
    )

## Run the pipeline

In [18]:
# Extract target variable
y = train_df.pop('Transported')

In [19]:
# Extract identifier column
i = train_df.pop('PassengerId')

## Split the data to create a validation set

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df, y, test_size = 0.25, random_state = 0)

## Preprocess the split data

In [21]:
preprocessor = preprocessor()
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)

## Instantiation of the models

In [22]:
classifiers = {
    'LogisticRegression': LogisticRegression(),
    'Perceptron': Perceptron(),
    'SVC': SVC(),
    'KNN5': KNeighborsClassifier(n_neighbors = 5),
    'KNN8': KNeighborsClassifier(n_neighbors = 8),
    'KNN10': KNeighborsClassifier(n_neighbors = 10),
    'KNN15': KNeighborsClassifier(n_neighbors = 15),
    'DecisionTree': DecisionTreeClassifier(),
    'Gaussian': GaussianNB(),
    'RandomForest': RandomForestClassifier(),
    'LinearSvc': LinearSVC(),
    'SGDClassifier': SGDClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'CatBoost': CatBoostClassifier(verbose=0),
}

In [23]:
# Add some more models to the classifiers dictionary
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

classifiers.update({
    'GradientBoosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'LightGBM': LGBMClassifier(),
    'SVC_linear': SVC(kernel='linear'),
    'SVC_poly': SVC(kernel='poly'),
    'SVC_sigmoid': SVC(kernel='sigmoid'),
})

## Train the models

### Define scoring function

In [24]:
def score_model(classifier, X_train, X_val, y_train, y_val):
    # Fit the model
    classifier.fit(X_train, y_train)

    # Predict the test data
    y_pred = classifier.predict(X_val)

    # Create a confusion matrix
    cm = confusion_matrix(y_val, y_pred)

    # Calculate the accuracy
    acc = accuracy_score(y_val, y_pred)

    scores = cross_val_score(classifier, X_train, y_train, cv = 10)

    return { 
            'confusion_matrix': cm,
            'accuracy': acc,
            'kfold-cv': scores.mean()
        }

def run_classifiers(classifiers: dict, X_train, X_test, y_train, y_val) -> dict:
    results = {}
    for name, classifier in classifiers.items():
        results[name] = score_model(classifier, X_train, X_test, y_train, y_val)
    return results

### Run the models

In [25]:
results = run_classifiers(classifiers, X_train, X_val, y_train, y_val)

[LightGBM] [Info] Number of positive: 3281, number of negative: 3238
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001019 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1603
[LightGBM] [Info] Number of data points in the train set: 6519, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503298 -> initscore=0.013192
[LightGBM] [Info] Start training from score 0.013192
[LightGBM] [Info] Number of positive: 2952, number of negative: 2915
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000559 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1603
[LightGBM] [Info] Number of data points in the train set: 5867, number of used features: 39
[LightGBM] [Info] [binary:

In [26]:
# convert results to a dataframe
results_df = pd.DataFrame(results).T
results_df.sort_values(by='kfold-cv', ascending=False)

,confusion_matrix,accuracy,kfold-cv
CatBoost,"[[857, 220], [191, 906]]",0.810948,0.808251
LightGBM,"[[852, 225], [182, 915]]",0.812787,0.80641
XGBoost,"[[855, 222], [215, 882]]",0.798988,0.804109
GradientBoosting,"[[806, 271], [165, 932]]",0.799448,0.803189
RandomForest,"[[885, 192], [258, 839]]",0.793008,0.794139
AdaBoost,"[[811, 266], [175, 922]]",0.797148,0.790304
DecisionTree,"[[786, 291], [276, 821]]",0.73919,0.740451
Gaussian,"[[805, 272], [322, 775]]",0.726771,0.734317
LogisticRegression,"[[898, 179], [412, 685]]",0.728151,0.728947
SVC_linear,"[[619, 458], [222, 875]]",0.687213,0.632315


## Tuning the best models

In [27]:
# Tune a model using GridSearchCV
def tune_model(classifier, param_grid):
    print(f'\n\nTuning {classifier.__class__.__name__}...')
    grid_search = GridSearchCV(classifier, param_grid, cv=10, n_jobs = -1, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_estimator = grid_search.best_estimator_
    score = score_model(best_estimator, X_train, X_val, y_train, y_val)

    print(f'Best SVC Accuracy: { score["accuracy"] }')
    print(f'Best SVC KFold CV: { score["kfold-cv"] }')
    print('\nConfusion Matrix:')
    print_confusion_matrix(score['confusion_matrix'])
    
    f1 = f1_score(y_val, best_estimator.predict(X_val))
    # and finally the f1 score
    print(f'F1 Score: {f1}')

    return (grid_search.best_params_, best_estimator, f1)

# Print the confusion matrix with labels for TP, FP, TN, FN
def print_confusion_matrix(cm: np.ndarray):
    # True Positives
    TP = cm[1, 1]
    # True Negatives
    TN = cm[0, 0]
    # False Positives
    FP = cm[0, 1]
    # False Negatives
    FN = cm[1, 0]

    # Now we have extracted the values print as a grid
    print(f'True Positives: {TP} False Positives: {FP}')
    print(f'False Negatives: {FN} True Negatives: {TN}')

In [28]:
tuning_results = [
    tune_model(
        XGBClassifier(),
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
            'eval_metric': ['logloss'], 
            'use_label_encoder': [False]
        }, 
    ),
    tune_model(
        RandomForestClassifier(), 
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'criterion': ['gini', 'entropy']
        }
    ),
    tune_model(
        CatBoostClassifier(verbose=0),
        param_grid = {
            'iterations': [100, 200, 300],
            'depth': [3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
        }
    ),
    tune_model(
        LGBMClassifier(verbose=0),
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
        }
    ),
]



Tuning XGBClassifier...
Best SVC Accuracy: 0.8040478380864765
Best SVC KFold CV: 0.8082520520577121

Confusion Matrix:
True Positives: 902 False Positives: 231
False Negatives: 195 True Negatives: 846
F1 Score: 0.8089686098654709


Tuning RandomForestClassifier...
Best SVC Accuracy: 0.7594296228150874
Best SVC KFold CV: 0.7611607437354518

Confusion Matrix:
True Positives: 756 False Positives: 182
False Negatives: 341 True Negatives: 895
F1 Score: 0.7429975429975431


Tuning CatBoostClassifier...
Best SVC Accuracy: 0.8008279668813247
Best SVC KFold CV: 0.8087114679633975

Confusion Matrix:
True Positives: 917 False Positives: 253
False Negatives: 180 True Negatives: 824
F1 Score: 0.8089986766651963


Tuning LGBMClassifier...
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

In [29]:
# Print out the F1 Scores for the different models
for result in tuning_results:
    print(f'F1 Score for {result[1].__class__}: {result[2]}')

F1 Score for <class 'xgboost.sklearn.XGBClassifier'>: 0.8089686098654709
F1 Score for <class 'sklearn.ensemble._forest.RandomForestClassifier'>: 0.7429975429975431
F1 Score for <class 'catboost.core.CatBoostClassifier'>: 0.8089986766651963
F1 Score for <class 'lightgbm.sklearn.LGBMClassifier'>: 0.8130298973672468


In [30]:
best_result = max(tuning_results, key=lambda x: x[2])[1]

## Create submission

Now we can predict the test set and submit

In [31]:
# Convert any Boolean columns with Unknown to String type
for column in test_df.columns:
    if 'Unknown' in test_df[column].unique():
        print(column)
        test_df[column] = test_df[column].astype(str)

HomePlanet
CryoSleep
Destination
Deck
Side


In [32]:
# Check the shape

print(f'Test: {test_df.shape}')

# Print the first 5 rows:
test_df.head()

Test: (4277, 15)


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side,Group,GroupSize
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,S,0013,1
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,S,0018,1
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,C,S,0019,1
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,C,S,0021,1
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,F,S,0023,1


In [33]:
# Capture the passenger IDs for submission and remove from the data as not useful predictors
idx_test = test_df.pop('PassengerId')

In [34]:
# Pre-process test set
X_test = preprocessor.transform(test_df) # sparse to dense array

In [35]:
# Predict the test set
y_pred = best_result.predict(X_test).astype(bool)

In [37]:
# Write the submission DataFrame
model='eighth'
submission = pd.DataFrame({'PassengerId': idx_test, 'Transported': y_pred})
submission.to_csv(f'submissions/{model}-submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!
